# Libraries

In [1]:
import os
import pandas as pd
import re
from itertools import chain
import numpy as np

# Read Parquet Files

In [2]:
folder_path_contracts = "19_DataAfterCleaning"

In [3]:
print('Contracts Files')
parquet_files_contracts = [f for f in os.listdir(folder_path_contracts) if f.endswith('.parquet')]
for f in parquet_files_contracts:
    print(f)

Contracts Files
csv_resultados_2015.parquet
csv_resultados_2016.parquet
csv_resultados_2017.parquet
csv_resultados_2018.parquet
csv_resultados_2019.parquet
csv_resultados_2020.parquet
csv_resultados_2021.parquet
csv_resultados_2022.parquet
csv_resultados_2023.parquet
csv_resultados_2024.parquet


## Dictionaries with all parquet files

In [4]:
dataframes_contracts = {}
for file in parquet_files_contracts:
    file_path = os.path.join(folder_path_contracts, file)
    df = pd.read_parquet(file_path)
    dataframes_contracts[file] = df

## Verify uploaded files

In [5]:
print("Arquivos carregados contratos:")
for i, name in enumerate(dataframes_contracts.keys(), start=1):
    print(f"{i}. {name}")

Arquivos carregados contratos:
1. csv_resultados_2015.parquet
2. csv_resultados_2016.parquet
3. csv_resultados_2017.parquet
4. csv_resultados_2018.parquet
5. csv_resultados_2019.parquet
6. csv_resultados_2020.parquet
7. csv_resultados_2021.parquet
8. csv_resultados_2022.parquet
9. csv_resultados_2023.parquet
10. csv_resultados_2024.parquet


## Aggregate parquet files

In [6]:
df_list_contracts = []
for filename, df in dataframes_contracts.items():
    df['Fonte_Arquivo'] = filename  # Adiciona coluna com o nome do ficheiro
    df_list_contracts.append(df)
df_geral_contratos = pd.concat(df_list_contracts, ignore_index=True)

# Basic Analysis

## Number of rows

In [7]:
print(f"\nNúmero total de linhas contratos: {df_geral_contratos.shape[0]}")


Número total de linhas contratos: 1575991


# Filtragem por objeto de contrato

## Dicionário de termos

In [ ]:
temas_cpv = {
    "saude": [
        "saúde", "cuidados", "tratamento", "clínico", "médico", "medico", "hospitalar", 
        "ambulatório", "terapêutico", "enfermagem", "reabilitação", "psiquiatria"
    ],
    "instituicoes_saude": [
        "hospital", "centro de saúde", "clínica", "unidade de saúde", 
        "unidade hospitalar", "instituição hospitalar"
    ],
    "profissionais_saude": [
        "médico", "enfermeiro", "técnico de saúde", "psicólogo", "fisioterapeuta", 
        "nutricionista", "cirurgião"
    ],
    "urgencia_emergencia": [
        "urgência", "emergência", "transporte de doentes", 
        "ambulância", "serviço de urgência"
    ],
    "outros_servicos": [
        "serviços médicos", "serviços de enfermagem", "serviços hospitalares", 
        "serviços clínicos", "apoio domiciliário", "cuidados continuados", 
        "cuidados paliativos", "serviços de diagnóstico", "laboratório", 
        "radiologia", "análises clínicas"
    ],
    "servicos_saude_externos": [
        "aquisição de serviços de saúde",
        "prestação de serviços de saúde",
        "serviços médicos externos",
        "serviços de enfermagem externos",
        "outsourcing de profissionais de saúde",
        "subcontratação de médicos",
        "subcontratação de enfermeiros",
        "fornecimento de médicos",
        "fornecimento de enfermeiros",
        "prestadores externos de serviços de saúde",
        "apoio médico externo",
        "apoio clínico externo",
        "pessoal médico temporário",
        "colocação de médicos",
        "colocação de pessoal clínico",
        "contratação externa de profissionais de saúde",
        "recursos humanos para serviços de saúde",
        "empresas de trabalho temporário na saúde",
        "prestação de cuidados médicos por terceiros"
    ]
}

## Filtragem

In [9]:
def filtrar_por_temas(df, temas_cpv, colunas_busca):
    df_resultados = []
    estatisticas = []
    for tema, termos in temas_cpv.items():
        # Cria regex com OR entre termos
        regex_termos = r'|'.join([re.escape(t.lower()) for t in termos])
        # Filtra linhas que contêm algum termo em qualquer das colunas de busca
        mask = df[colunas_busca].apply(lambda col: col.astype(str).str.lower().str.contains(regex_termos)).any(axis=1)
        df_tema = df[mask].copy()
        df_tema['tema_cpv'] = tema
        # Armazena resultados
        df_resultados.append(df_tema)
        # Estatísticas por CPV
        contagem_cpv = df_tema['cpv_number'].value_counts().reset_index()
        contagem_cpv.columns = ['cpv_number', 'total']
        contagem_cpv['tema_cpv'] = tema
        estatisticas.append(contagem_cpv)
        # Amostra
        print(f"\n=== {tema.upper()} ===")
        with pd.option_context('display.max_colwidth', None):
            display(df_tema[['objectoContrato', 'cpv_number', 'adjudicante_description']].head(40))
        print(f"Total de ocorrências: {len(df_tema)}")
    df_filtrado = pd.concat(df_resultados, ignore_index=True)
    df_estatisticas = pd.concat(estatisticas, ignore_index=True)
    return df_filtrado, df_estatisticas

In [10]:
colunas_busca = ['objectoContrato', 'adjudicante_description']
# Para os contratos que contêm algum CPV válido
df_in_filtrado, stats_in = filtrar_por_temas(df_geral_contratos, temas_cpv, colunas_busca)
print(f"\nNúmero total de linhas contratos com termos do dicionário: {df_in_filtrado.shape[0]}")


=== SAUDE ===


,objectoContrato,cpv_number,adjudicante_description
10,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
11,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
12,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
13,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
14,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
15,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
16,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
17,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
18,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
19,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"


Total de ocorrências: 448974

=== INSTITUICOES_SAUDE ===


,objectoContrato,cpv_number,adjudicante_description
10,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
11,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
12,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
13,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
14,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
15,"Derivados do Plasma, AQ 2012/09",[33600000-6],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
16,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
17,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
18,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"
19,"Seringas e Contentores, AQ 2012/22 (Renov 2020014)",[33100000-1],"[Centro Hospitalar Vila Nova de Gaia - Espinho, E. P. E.]"


Total de ocorrências: 313851

=== PROFISSIONAIS_SAUDE ===


...

Total de ocorrências: 11724

=== OUTROS_SERVICOS ===


,objectoContrato,cpv_number,adjudicante_description
396,"Aquisição de máquinas, aparelhos e ferramentas para ensaio e medição para o Laboratório de Ensaio e Caracterização e para o Laboratório de Interface e Robótica - lote A",[38540000-2],[Instituto Politécnico do Cávado e do Ave]
397,"Aquisição de máquinas, aparelhos e ferramentas para ensaio e medição para o Laboratório de Ensaio e Caracterização e para o Laboratório de Interface e Robótica - lote B",[38540000-2],[Instituto Politécnico do Cávado e do Ave]
398,"Aquisição de máquinas, aparelhos e ferramentas para ensaio e medição para o Laboratório de Ensaio e Caracterização e para o Laboratório de Interface e Robótica - lote C",[38540000-2],[Instituto Politécnico do Cávado e do Ave]
401,"Prestação de serviços médicos, que incide na execução de atos médicos para acompanhamento dos atletas em situação de enquadramento técnico na DESMOR",[85121000-3],"[Desmor, E. M., S. A.]"
726,Aquisição de equipamentos de simulação para investigação para o Laboratório de Jogos Digitais e para o Laboratório de Interface e Robótica,[38970000-5],[Instituto Politécnico do Cávado e do Ave]
789,"Fornecimento de equipamentos FTIR , UV-V para laboratório",[38000000-5],[Autoridade Tributária e Aduaneira]
884,Aquisição de seguro para a frota automóvel,[66514110-0],"[LNEG - Laboratório Nacional de Energia e Geologia, I. P.]"
1004,Serviços de enfermagem a desenvolver no Posto de Enfermagem do Centro Histórico da Vila de Campo Maior,[85141200-1],[Município de Campo Maior]
1005,Serviços de enfermagem a desenvolver no Posto de Enfermagem do Centro Histórico da Vila de Campo Maior,[85141200-1],[Município de Campo Maior]
1008,Serviços de enfermagem a desenvolver no Posto de Enfermagem do Centro Histórico da Vila de Campo Maior,[85141200-1],[Município de Campo Maior]


Total de ocorrências: 24787

=== SERVICOS_SAUDE_EXTERNOS ===


,objectoContrato,cpv_number,adjudicante_description
3208,Contrato para a “prestação de serviços de Saúde do Trabalho pelo período de 36 meses”,[85147000-1],[Município de Arganil]
9719,Prestação de serviços de saúde no trabalho,[71317200-5],[CIMT - Comunidade Intermunicipal do Médio Tejo]
10275,Aquisição de serviços de saúde no trabalho,[85000000-9],[Município de Góis]
13015,"Prestação de Serviços de Saúde no Trabalho, enquadrado no Lote 1 do Acordo Quadro para Prestação de Serviços de Segurança, Higiene e Saúde no Trabalho, celebrado entre a Comunidade Intermunicipal da Região de Leiria (doravante designada por CIMRL) e as entidades prestadoras de serviços previamente selecionadas para prestação dos serviços descritos no lote 1",[85147000-1],[Município de Figueiró dos Vinhos]
16751,Aquisição de serviços de saúde para o 1º trimestre de 2015_Ifo. 70/DCP/2015,[85000000-9],[Direção-Geral de Reinserção e Serviços Prisionais]
20116,Aquisição de serviços de Saúde no Trabalho,[85147000-1],"[Suldouro Valorização e Tratamento de Resíduos Sólidos Urbanos, S. A.]"
22515,Aquisição de serviços de saúde no trabalho 2015 - 2017,[71317200-5],"[Infralobo - Empresa de Infraestruturas de Vale do Lobo, E.M.]"
30117,PE_15046_ADS - PRESTAÇÃO DE SERVIÇOS DE SAÚDE PARA OS TRABALHADORES DA PARQUE ESCOLAR,[98390000-3],"[Parque Escolar, E. P. E.]"
32113,PRESTAÇÃO DE SERVIÇOS DE SAÚDE NO TRABALHO PARA O MUNICÍPIO DE VILA DO CONDE,[85100000-0],[Município de Vila do Conde]
40241,Procedimento para aquisição de serviços de saúde_PAD/0083/DCP/2015,[85000000-9],[Direção-Geral de Reinserção e Serviços Prisionais]


Total de ocorrências: 469

Número total de linhas contratos com termos do dicionário: 816640


# Import CPV file

In [11]:
input_folder = r"16_CPV"
file = 'TiposContrato_CPV_TED_V1.0_filtered.parquet'
cpv_file_path = os.path.join(input_folder, file)

In [12]:
df_cpvs = pd.read_parquet(cpv_file_path)
print(df_cpvs.columns)

Index(['ID', 'Tipo de Contrato', 'CPV', 'Ativo', 'CPVcode', 'CPVdescription'], dtype='object')


In [13]:
pd.set_option('display.max_columns', None) 
display(df_cpvs.head(3))

,ID,Tipo de Contrato,CPV,Ativo,CPVcode,CPVdescription
0,7690,Aquisição de serviços,73000000-2 - Serviços de investigação e desenv...,1,73000000-2,Serviços de investigação e desenvolvimento e s...
1,7691,Aquisição de serviços,73100000-3 - Serviços de desenvolvimento exper...,1,73100000-3,Serviços de desenvolvimento experimental e de ...
2,7692,Aquisição de serviços,73110000-6 - Serviços de investigação,1,73110000-6,Serviços de investigação


In [14]:
print(f"\nNúmero total de linhas CPV: {df_cpvs.shape[0]}")


Número total de linhas CPV: 222


In [15]:
# List with unique cpv values from the manual filtering
unique_cpvs_manual = df_cpvs['CPVcode'].dropna().unique().tolist()

In [16]:
print("Number of unique CPVs:", len(unique_cpvs_manual))
print("Sample of unique CPVs:", unique_cpvs_manual[:10])

Number of unique CPVs: 112
Sample of unique CPVs: ['73000000-2', '73100000-3', '73110000-6', '73111000-3', '73120000-9', '73200000-4', '73210000-7', '73220000-0', '73300000-5', '75000000-6']


# Crossing CPV filtered manually with the CPV by dictionary

In [17]:
print(f"\nNúmero total de linhas: {df_in_filtrado.shape[0]}")


Número total de linhas: 816640


In [18]:
display(df_in_filtrado.head(3))

,idcontrato,nAnuncio,TipoAnuncio,idINCM,tipoContrato,idprocedimento,tipoprocedimento,objectoContrato,descContrato,adjudicante,adjudicatarios,dataPublicacao,dataCelebracaoContrato,precoContratual,cpv,prazoExecucao,localExecucao,fundamentacao,ProcedimentoCentralizado,numAcordoQuadro,DescrAcordoQuadro,precoBaseProcedimento,dataDecisaoAdjudicacao,dataFechoContrato,PrecoTotalEfetivo,regime,justifNReducEscrContrato,tipoFimContrato,CritMateriais,concorrentes,linkPecasProc,Observacoes,ContratEcologico,Ano,fundamentAjusteDireto,adjudicante_string,adjudicante_nipc,adjudicante_description,adjudicatarios_string,adjudicatarios_nipc,adjudicatarios_description,dataPublicacaoParsed,anoPublicação,dataCelebracaoContratoParsed,anoDataCelebracaoContrato,precoContratualInterval,cpv_string,cpv_number,cpv_description,prazoExecucaoIntervalo,precoBaseProcedimentoInterval,dataDecisaoAdjudicacaoParsed,anoDataDecisaoAdjudicacao,dataFechoContratoParsed,anoDataFechoContrato,PrecoTotalEfetivoInterval,concorrentes_string,concorrentes_nipc,concorrentes_description,Fonte_Arquivo,tema_cpv
0,1336740,,,<NA>,[Aquisição de bens móveis],1335213,Ao abrigo de acordo-quadro (art.º 259.º),"Derivados do Plasma, AQ 2012/09",Baxter,[508142156 - Centro Hospitalar Vila Nova de Ga...,"[503347345 - BAXTER MEDICO - FARMACÊUTICA, LDA]",02/01/2015,02/01/2015,48690.0,[33600000-6 - Produtos farmacêuticos],365,"[Portugal, Porto, Vila Nova de Gaia]",Artigo 259.º do Código dos Contratos Públicos,False,535788,CONCURSO PÚBLICO PARA A CELEBRAÇÃO DE CONTRATO...,1077702.6,06/11/2014,,0.0,Código dos Contratos Públicos (DL 18/2008),<NA>,,False,None,,,False,2015,,['508142156 - Centro Hospitalar Vila Nova de G...,[508142156],[Centro Hospitalar Vila Nova de Gaia - Espinho...,"['503347345 - BAXTER MEDICO - FARMACÊUTICA, LDA']",[503347345],"[BAXTER MEDICO - FARMACÊUTICA, LDA]",2015-01-02,2015,2015-01-02,2015.0,"5,001 € to 50,000 €",['33600000-6 - Produtos farmacêuticos'],[33600000-6],[Produtos farmacêuticos],360-389 dias,"139,001 € to 5,350,000 €",2014-11-06,2014.0,NaT,NaN,zero,<NA>,None,None,csv_resultados_2015.parquet,saude
1,1336741,,,<NA>,[Aquisição de bens móveis],1335213,Ao abrigo de acordo-quadro (art.º 259.º),"Derivados do Plasma, AQ 2012/09",CSL Behring,[508142156 - Centro Hospitalar Vila Nova de Ga...,"[503047201 - CSL Behring, Lda]",02/01/2015,02/01/2015,600607.5,[33600000-6 - Produtos farmacêuticos],365,"[Portugal, Porto, Vila Nova de Gaia]",Artigo 259.º do Código dos Contratos Públicos,False,535788,CONCURSO PÚBLICO PARA A CELEBRAÇÃO DE CONTRATO...,1077702.6,06/11/2014,,0.0,Código dos Contratos Públicos (DL 18/2008),<NA>,,False,None,,,False,2015,,['508142156 - Centro Hospitalar Vila Nova de G...,[508142156],[Centro Hospitalar Vila Nova de Gaia - Espinho...,"['503047201 - CSL Behring, Lda']",[503047201],"[CSL Behring, Lda]",2015-01-02,2015,2015-01-02,2015.0,"139,001 € to 5,350,000 €",['33600000-6 - Produtos farmacêuticos'],[33600000-6],[Produtos farmacêuticos],360-389 dias,"139,001 € to 5,350,000 €",2014-11-06,2014.0,NaT,NaN,zero,<NA>,None,None,csv_resultados_2015.parquet,saude
2,1336742,,,<NA>,[Aquisição de bens móveis],1335213,Ao abrigo de acordo-quadro (art.º 259.º),"Derivados do Plasma, AQ 2012/09",Grifols,[508142156 - Centro Hospitalar Vila Nova de Ga...,"[502041285 - GRIFOLS PORTUGAL, LDA]",02/01/2015,02/01/2015,281238.6,[33600000-6 - Produtos farmacêuticos],365,"[Portugal, Porto, Vila Nova de Gaia]",Artigo 259.º do Código dos Contratos Públicos,False,535788,CONCURSO PÚBLICO PARA A CELEBRAÇÃO DE CONTRATO...,1077702.6,06/11/2014,,0.0,Código dos Contratos Públicos (DL 18/2008),<NA>,,False,None,,,False,2015,,['508142156 - Centro Hospitalar Vila Nova de G...,[508142156],[Centro Hospitalar Vila Nova de Gaia - Espinho...,"['502041285 - GRIFOLS PORTUGAL, LDA']",[502041285],"[GRIFOLS PORTUGAL, LDA]",2015-01-02,2015,2015-01-02,2015.0,"139,001 € to 5,350,000 €",['33600000-6 - Produtos farmacêuticos'],[33600000-6],[Produtos farmacêuticos],360-389 dias,"139,001 € to 5,350,000 €",2014-11-06,2

In [32]:
def extract_cpvs(entry):
    if entry is None or entry == 'None':
        return []
    if isinstance(entry, list) or isinstance(entry, np.ndarray):
        return list(chain.from_iterable(extract_cpvs(e) for e in entry))
    if isinstance(entry, str):
        # Extract all CPV-like codes using regex
        return re.findall(r'\d{8}-\d', entry)
    return []

# Apply extraction
all_cpvs_extracted = df_in_filtrado['cpv_number'].dropna().apply(extract_cpvs)

# Flatten and deduplicate
unique_cpvs_dictionary = sorted(set(chain.from_iterable(all_cpvs_extracted)))

# Display result
print("Number of unique CPV codes:", len(unique_cpvs_dictionary))
print("Sample CPV codes:", unique_cpvs_dictionary[:10])

Number of unique CPV codes: 4341
Sample CPV codes: ['03000000-1', '03100000-2', '03110000-5', '03111000-2', '03114000-3', '03115110-4', '03117100-5', '03121000-5', '03121100-6', '03131100-9']


In [33]:
# Convert to sets for fast set operations
set_manual = set(unique_cpvs_manual)
set_dictionary = set(unique_cpvs_dictionary)
# CPVs in both lists
cpvs_common = sorted(set_manual & set_dictionary)
# CPVs only in manual/reference list
cpvs_only_in_manual = sorted(set_manual - set_dictionary)
# CPVs only in dictionary/contratos list
cpvs_only_in_dictionary = sorted(set_dictionary - set_manual)
# Show results
print("CPVs in both lists:", len(cpvs_common))
print("CPVs only in manual:", len(cpvs_only_in_manual))
print("CPVs only in dictionary:", len(cpvs_only_in_dictionary))
# Optional: preview samples
print("\nSample common CPVs:", cpvs_common[:10])
print("Sample only in manual:", cpvs_only_in_manual[:10])
print("Sample only in dictionary:", cpvs_only_in_dictionary[:10])

CPVs in both lists: 101
CPVs only in manual: 11
CPVs only in dictionary: 4240

Sample common CPVs: ['73000000-2', '73100000-3', '73110000-6', '73111000-3', '73120000-9', '73200000-4', '73210000-7', '73220000-0', '73300000-5', '75000000-6']
Sample only in manual: ['85111310-6', '85111600-6', '85121271-3', '85131110-0', '85141100-0', '85142200-8', '85142400-0', '85146000-4', '85146100-5', '85146200-6']
Sample only in dictionary: ['03000000-1', '03100000-2', '03110000-5', '03111000-2', '03114000-3', '03115110-4', '03117100-5', '03121000-5', '03121100-6', '03131100-9']


In [34]:
input_folder = r"16_CPV"
file = 'TiposContrato_CPV_TED_V1.0_new.parquet'
cpv_file_path = os.path.join(input_folder, file)

In [35]:
df_all_cpvs = pd.read_parquet(cpv_file_path)

In [36]:
# print the cpv codes only in manual
pd.set_option('display.max_columns', None) 
df_cpvs_only_in_manual = df_all_cpvs[df_all_cpvs['CPVcode'].isin(cpvs_only_in_manual)]
# Display the full set of columns for these rows
display(df_cpvs_only_in_manual[['Tipo de Contrato','CPVcode','CPVdescription']])

,Tipo de Contrato,CPVcode,CPVdescription
8154,Aquisição de serviços,85111310-6,Serviços de fertilização in vitro
8158,Aquisição de serviços,85111600-6,Serviços de ortoterapia
8181,Aquisição de serviços,85121271-3,Serviços ao domicílio para pessoas com perturb...
8193,Aquisição de serviços,85131110-0,Serviços cirúrgicos de ortodôncia
8196,Aquisição de serviços,85141100-0,Serviços prestados por parteiras
8203,Aquisição de serviços,85142200-8,Serviços de homeopatia
8205,Aquisição de serviços,85142400-0,Entrega ao domicílio de produtos para incontin...
8210,Aquisição de serviços,85146000-4,Serviços prestados por bancos de sangue
8211,Aquisição de serviços,85146100-5,Serviços prestados por bancos de esperma
8212,Aquisição de serviços,85146200-6,Serviços prestados por bancos de órgãos


In [37]:
# print the cpv codes only in dictionary
pd.set_option('display.max_columns', None) 
df_cpvs_only_in_dictionary = df_all_cpvs[df_all_cpvs['CPVcode'].isin(cpvs_only_in_dictionary)]
# Display the full set of columns for these rows
display(df_cpvs_only_in_dictionary[['Tipo de Contrato','CPVcode','CPVdescription']])

,Tipo de Contrato,CPVcode,CPVdescription
0,Aquisição de bens móveis,03000000-1,"Produtos da agricultura, da pesca, da silvicul..."
1,Aquisição de bens móveis,03100000-2,Produtos agrícolas e hortofrutícolas
2,Aquisição de bens móveis,03110000-5,"Cereais, produtos de culturas industriais e da..."
3,Aquisição de bens móveis,03111000-2,Sementes
17,Aquisição de bens móveis,03114000-3,Palha e plantas forrageiras
...,...,...,...
30419,Sociedade,98394000-1,Serviços de estofamento
30420,Sociedade,98395000-8,Serviços de serralharia
30421,Sociedade,98396000-5,Serviços de afinação de instrumentos
30430,Sociedade,98513310-8,Serviços de assistência ao domicílio


In [38]:
# Ensure all cpv_number entries are lists
df_in_filtrado['cpv_number'] = df_in_filtrado['cpv_number'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else ([x] if not isinstance(x, list) else x)
)
# Define a filtering function
def has_cpv_from_only_in_dictionary(cpv_list):
    return any(cpv in cpvs_only_in_dictionary for cpv in cpv_list)
# Apply the filter
filtered_df = df_in_filtrado[df_in_filtrado['cpv_number'].apply(has_cpv_from_only_in_dictionary)]
# Display a sample of the result
display(filtered_df[['cpv_number', 'objectoContrato', 'adjudicante_description']].head(10))


,cpv_number,objectoContrato,adjudicante_description
